In [1]:
import os
import tiktoken 
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain import OpenAI

if __name__ == "__main__":
    print("hello pdf")
    
    #load env.key
    load_dotenv()
    
    # Access the API key
    openai_api_key = os.getenv('OPENAI_API_KEY')


    pdf_path = r"C:\Users\(Ai)AiSukmoren\Desktop\intro-to-vector-db\react.pdf"
    loader = PyPDFLoader(file_path=pdf_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
    docs = text_splitter.split_documents(documents=documents)
    #chunk unit
    docs = len(docs)
    print("the unit of chunck is {docs}")
    
    embeddings = OpenAIEmbeddings()
    vectorstores = FAISS.from_documents(docs,embeddings)
    vectorstores.save_local("faiss_index_react")
    
    new_vectorstore = FAISS.load_local("faiss_index_react",embeddings)
    
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type="stuff", retriever=new_vectorstore.as_retriever(),
    )
    
    result = qa.run("Give me the gist of ReAct in 3 sentences")
    print(result)

hello pdf


ImportError: pypdf package not found, please install it with `pip install pypdf`